1) Se cuenta con los siguientes archivos con el registro de la
participación de los distintos jugadores en la liga nacional de fútbol.

Jugador (ID_JUGADOR, Nombre_apellido, fecha_nacimiento, país)

Estadísticas (ID_PARTIDO, ID_JUGADOR, tiempo_juego, distancia_recorrida, goles)

Partido (ID_PARTIDO, año, semestre, nro_fecha, equipo_local, equipo_visitante)

Utilizando el API de RDD de PySpark, se desea obtener un RDD con
el nombre y apellido de los jugadores nacidos en Argentina que
hayan aumentado en más de un 50% el tiempo de juego total de
un semestre al otro. Cada año se juegan dos campeonatos (uno por
semestre) y se deben comparar semestres consecutivos (por ejemplo
2017-2 con 2018-1, o 2018-1 con 2018-2). El tiempo de juego total
para un campeonato es la suma de todo el tiempo jugado en los
partidos de ese campeonato.
(****) ( 20pts)

In [25]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

matchesRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2019-2C-1OpMatches.csv', header=True).rdd
playersRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2019-2C-1OpPlayers.csv', header=True).rdd
statsRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2019-2C-1OpStats.csv', header=True).rdd

matchesRdd = matchesRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), pd.to_numeric(x[1], 'coerce')
                                       , pd.to_numeric(x[2], 'coerce'), pd.to_numeric(x[3], 'coerce')
                                       , x[4], x[5]))

statsRdd = statsRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), pd.to_numeric(x[1], 'coerce')
                                       , pd.to_numeric(x[2], 'coerce'), pd.to_numeric(x[3], 'coerce')
                                       , pd.to_numeric(x[4], 'coerce')))

playersRdd = playersRdd.map(lambda x: (pd.to_numeric(x[0], 'coerce'), x[1], x[2], x[3]))

playersFromArgentina = playersRdd.filter(lambda x: x[3] == 'Argentina').map(lambda x: (x[0], x[1]))
playersStats = statsRdd.map(lambda x: (x[1], (x[0], x[2])))
playersFromArgStats = playersStats.join(playersFromArgentina).map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))

minutesPlayedPerSeasonPerPlayer = playersFromArgStats.join(matchesRdd.map(lambda x: (x[0], (x[1], x[2])))) \
                                .map(lambda x: ((x[1][0][0], x[1][1][0], x[1][1][1]), x[1][0][1])) \
                                .reduceByKey(lambda x,y: x + y)

def f(x):
    if x[0][2] == 1:
        return (x[0], (x[1], None)), ((x[0][0], x[0][1]-1, 2), (None, x[1]))
    else:
        return (x[0], (x[1], None)), ((x[0][0], x[0][1], 1), (None, x[1]))

minutesPlayedPerSeasonPerPlayer.flatMap(f).reduceByKey(lambda x,y: (x[0] if (y[0] is None) else y[0],
                                                                    y[1] if (x[1] is None) else x[1])) \
        .filter(lambda x: (x[1][0] is not None) & (x[1][1] is not None)) \
        .filter(lambda x: abs(x[1][0] - x[1][1])/x[1][0] > 0.5).map(lambda x: x[0][0]).distinct().collect()

['Danby',
 'Shelba',
 'Tjon',
 'Sikorski',
 'Saree',
 'Bashemeth',
 'Smitt',
 'Rugen',
 'Ricki',
 'Fadiman',
 'Thomasina',
 'Albertine',
 'Colleen',
 'Dulciana',
 'Lymann',
 'Goode',
 'Adore',
 'Burkle',
 'Hathaway',
 'Buffum',
 'Barrus',
 'Noman',
 'Ambrosia',
 'Brotherson',
 'Wenda',
 'Carleen',
 'Buckler',
 'Mullane',
 'Millda',
 'Bari',
 'Scammon',
 'Karl',
 'Chapland',
 'Posner',
 'Rolf',
 'Noam',
 'Thunell',
 'Wiener',
 'Vilma',
 'Wandie',
 'Ashely',
 'Dalli',
 'Jagir',
 'Hachmin',
 'Argus',
 'Barney',
 'Chesna',
 'Selway',
 'Faso',
 'Tillford',
 'Lea',
 'Wilona',
 'Sisile',
 'Wenoa',
 'Dyche',
 'Trace',
 'Weaks',
 'Constancy',
 'Helfand',
 'Vorster',
 'Stav',
 'Rheingold',
 'Christal',
 'Gladstone',
 'Publia',
 'Mozelle',
 'Anastatius',
 'Jacinda',
 'Florina',
 'Andrel',
 'Simmonds',
 'Leffen',
 'Carmena',
 'Gaulin',
 'Nore',
 'Bebe',
 'Bultman',
 'Gino',
 'Dearborn',
 'Medrek',
 'Hepsibah',
 'Ajay',
 'Klemperer',
 'Sparhawk',
 'Chandler',
 'Clie',
 'Hillel',
 'Henrie']